In [ ]:
%reload_ext autoreload
%autoreload 2

En este archivo puedes escribir lo que estimes conveniente. Te recomendamos detallar tu solución y todas las suposiciones que estás considerando. Aquí puedes ejecutar las funciones que definiste en los otros archivos de la carpeta src, medir el tiempo, memoria, etc.

# **Consideraciones iniciales**

Antes de comenzar con el challenge, es importante la selección de herramientas que permitirirán aprovechar de mejor manera los recurso, para esta ocasión se hace uso de un conjunto de herramientas ofrecidas por Google Cloud, las cuales se detallan a continuación:



1.  **Google Drive**: Se hace uso para almacenar y sincronizar los archivos del proyecto.y facilitar la integración con otras herramientas.
2.   **GitHub y Git Flow**: HSe ha configurado un repositorio en GitHub y se realiza la gestión del flujo trabajo con Git Flow permitiendo mantener un historial de los cambios y control de versiones.
3.   **Google Cloud Storage (Bucket):** Para el almacenamiento del archivo JSON que contiene los tweets, se hace uso de Google Cloud Storage, esto facilita el acceso de otras herrmaientas con BigQuery permitiendo optimizar el análisis y procesamiento de los datos.
4.   **BigQuery**: se emplea BigQuery para realizar consultas SQL sobre el conjunto de datos de tweets, donde gracias a su capacidad de procesar grandes volúmenes de datos, se optimiza tanto tiempo como memoria en el analísis de los datos.
5.   **Google Colab**: El uso de Google Colab permitirá ejecutar el cuaderno de Jupyter, así como también la integración con Google Drive y su capacidad de ejecución en la nube.

**NOTA:** Para maximizar la eficiencia en el almacenamiento y el procesamiento de datos en BigQuery,como mejoras en el proceso, se podría realizar una optimización del esquema y limpieza del datasets

### **Definición de variables globales**

Se definen las constantes se utilizan para configurar y controlar el desarrollo del challenge. Se utilizan principalmente variables para especificar rutas de archivos, nombres de archivos, nombres de proyectos y datasets en GCP:


*   Las variables **BUCKET_NAME, FOLDER_NAME, ZIP_FILE_NAME, FILE_ID y GCS_SOURCE_URI** almacenan información sobre el almacenamiento en Google Cloud Storage.
*   **MOUNT_POINT** y S**OURCE_PATH** especifican la ubicación en la que se montará Google Drive y la ruta de origen de los archivos.
*   **PROJECT_ID, DATASET_NAME** y **TABLE_NAME** contienen información sobre el proyecto en Google Cloud Platform y los detalles del dataset y tabla en BigQuery.

In [ ]:
# Google Cloud Storage (GCS) information
# Nombre del bucket en GCS
BUCKET_NAME: str = "tw-gcp-public-lab"
# Nombre de la carpeta en el bucket
FOLDER_NAME: str = "raw"
# Nombre del archivo ZIP de los tweets
ZIP_FILE_NAME: str = "tweets.json.zip"
# ID del archivo original en Google Drive (Link proporcionado)
FILE_ID: str = "1ig2ngoXFTxP5Pa8muXo02mDTFexZzsis"
# URI de origen en GCS
GCS_SOURCE_URI: str = f"gs://{BUCKET_NAME}/{FOLDER_NAME}/"

#Punto de montura para Google Drive
MOUNT_POINT = '/content/drive'
# Ruta de origen de los archivos
SOURCE_PATH = f'{MOUNT_POINT}/Othercomputers/My MacBook Pro/latam-challenge/'

# ID del proyecto en GCP
PROJECT_ID: str =  "tw-techdash"
# Nombre del dataset en BigQuery
DATASET_NAME: str = "tweets_dataset"
# Nombre de la tabla en BigQuery
TABLE_NAME: str = "tweets"

# Preparación del ambiente

Como se mencionó se hará uso de Google Drive como repositorio para la integración con GitHub y Google Colab, por lo que se realizan los siguientes pasos:
 1. Importar librerías necesarias para la correcta ejecución del notebook.
 2. Realizar el punto de montura hacia Google Drive para manipulación de los archivos.
 3. Instalación de dependencias establecidas en el archivo requirements.txt



In [ ]:
import os
import io                                             # Importar el módulo para manejar archivos en memoria
import zipfile                                        # Importar el módulo para trabajar con archivos ZIP
import subprocess
from google.cloud import storage                      # Importar el módulo del cliente de Google Cloud Storage
from google.colab import auth, drive                  # Importar el módulo de autenticación con Google Drive
from googleapiclient.discovery import build           # Importar el módulo para construir el cliente de Google Drive
from googleapiclient.http import MediaIoBaseDownload  # Importar el módulo para descargar archivos de Google Drive

def montar_google_drive():
    """
    Monta Google Drive en el entorno de Google Colab, si aún no está montado
    """
    if not os.path.isdir(MOUNT_POINT):
        drive.mount(MOUNT_POINT)
    else:
        print("Google Drive ya está montado")

def cambiar_directorio(ruta):
    """
    Cambia el directorio de trabajo actual
    """
    try:
        os.chdir(ruta)
        print(f'Directorio cambiado a: {os.getcwd()}')
    except FileNotFoundError:
        print(f'Director no encontrado: {ruta}')

def instalar_dependencias(ruta_archivo):
    """
    Instala las dependencias especificadas en el archivo requirements.txt
    """
    subprocess.check_call(['pip', 'install', '-r', ruta_archivo])

# Montar Google Drive si aún no está montado
montar_google_drive()

# Cambiar al directorio deseado
cambiar_directorio(f"{SOURCE_PATH}/src")

# Instalar las dependencias del proyecto
instalar_dependencias("../requirements.txt")

[autoreload of aux_functions failed: Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/usr/local/lib/python3.10/dist-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/usr/lib/python3.10/imp.py", line 315, in reload
    return importlib.reload(module)
  File "/usr/lib/python3.10/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 619, in _exec
  File "<frozen importlib._bootstrap_external>", line 883, in exec_module
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "/content/drive/Othercomputers/My MacBook Pro/latam-challenge/src/aux_functions.py", line 154, in <module>
    ) -> List[Tuple[any, any]]:
NameError: name 'List' is not defined
]


Google Drive ya está montado
Directorio cambiado a: /content/drive/Othercomputers/My MacBook Pro/latam-challenge/src


# Descarga del Archivo y Almacenamiento en Google Cloud Storage

En esta sección, se realiza la descarga del archivo de tweets en formato ZIP y se almacena en un Bucket de Google Cloud Storage (GCS). Este paso es esencial para su posterior análisis. Se ejecuta por única vez.

Para ello, se utilizan funciones auxiliares definidas en el archivo `aux_functions.py,` las cuales tienen métodos para la descarga, descompresión y almacenamiento de archivos en GCS.

El flujo es el siguiente:

* Descarga del archivo ZIP que contiene los tweets.
* Descomprimir el archivo ZIP para acceder a los datos de tweets en formato JSON.
* Almacenamiento del archivo JSON en un Bucket de Google Cloud Sotrage.

**NOTA**: Es importante mencionar que se utilizó este enfoque teninedo en cuenta la escalabilidad que se podría requerir en un escenario real.


In [ ]:
from aux_functions import *  # Importar funciones auxiliares para el desarrollo del challenge

#Inicializa un objeto de tipo BytesIO para almacenar el contenifo del archivo a descargar
downloaded= io.BytesIO()

try:
    authenticate_google_drive()  # Autentica con Google Drive
    drive_service= build('drive', 'v3')  # Crea un objeto de servicio de Google Drive

    # Acceso al bucket en GCS
    bucket = storage.Client().bucket(BUCKET_NAME)

    # Descarga del archivo original
    downloaded = download_file_from_drive(drive_service, FILE_ID)

    # Evalua si el archivo contiene data.
    if downloaded is None:
        print("Skipping upload and extraction as file already exists on cloud storage with matching size.")
    else:
        # Carga el archivo en GCS siempre y cuando tenga información
        uploaded_blob = upload_file_to_cloud_storage(bucket, FOLDER_NAME, downloaded, ZIP_FILE_NAME)

        # Descomprime el .ZIP, cuando aplique.
        if uploaded_blob.content_type == 'application/zip':
            json_file_name: str = decompress_zip_file(bucket, FOLDER_NAME, ZIP_FILE_NAME)

    print("File transfer successful!")

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    downloaded.close()
    print("File transfer process completed.")

Descargando 100%
Archivo subido a gs://tw-gcp-public-lab/raw/tweets.json.zip
File decompressed in gs://tw-gcp-public-lab/raw/farmers-protest-tweets-2021-2-4.json
File transfer successful!
File transfer process completed.


**Carga de datos en BigQuery**

Se realiza la carga de datos almacenados en Google Cloud storage hacia una tabla de BigQuery, para ello se valida que exista la tabla, caso contrario se la crea.

In [ ]:
# Authenticate to BigQuery
bigquery_client = authenticate_bigquery(PROJECT_ID)

# Create dataset (overwrite if needed)
create_dataset(bigquery_client, DATASET_NAME, mode='overwrite')

# Create table (overwrite if needed)
create_table(bigquery_client, DATASET_NAME, TABLE_NAME, mode='overwrite')

# Load data from Cloud Storage
load_data_from_storage(bigquery_client, GCS_SOURCE_URI, DATASET_NAME, TABLE_NAME, json_file_name)

print("Data loading completed!")

Dataset 'tweets_dataset' overwritten.
Table 'tweets' created.
Data loading completed!


# **Desarrollo**

Para el desarrollo del Challenge, se utilizará Bigquery para la resolución de las tres preguntas planteadas, por sus capacidad para procesar datos a gran escala, dentro de los beneficios que nos ofrece, se puede mencionar:
* Es Escalable y puede manejar grandes volumnes de datos.
* Estça diseñado para ofrecer resultados de consultas rápidos, gracias a su motor de consultas distribuidas.
* Permita integrarse fácilmente con Google Cloud Storage para el almacenamiento de los datos.

**DISCLAIMER POST-ENVÍO:**

Se ha utilizado el mismo query tanto para optimizar la memoria como el tiempo de ejecución, debido a las capacidades de BigQuery, ya que ha proporcionado eficiencia en ambos aspectos, logrando obtener resultados satisfactorios en las consultas.

Como futuras mejoras, se podría considerar la implementación de particiones en las tablas o el uso de caché si estos resultados se utilizarían frecuentemente. Ya que nos porían ayudar a mejorar aún más el rendimiento y la eficiencia de nuestras consultas, especialmente en casos de conjuntos de datos más grandes o consultas recurrentes.

Para esta ocasión, se utilizó expresiones comunes de tabla (CTE), uniones (JOINS), y agregaciones (AGG).

## **Question 01**: Top 10 fechas con top usuarios

In [ ]:
%run q1_memory

In [ ]:
from q1_memory import q1_memory
q1_memory(bigquery_client)


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/local/lib/python3.10/dist-packages/memory_profiler.py", line 847, in enable
    sys.settrace(self.trace_memory_usage)


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/local/lib/python3.10/dist-packages/memory_profiler.py", line 850, in disable
    sys.settrace(self._original_trace_function)



Filename: /content/drive/Othercomputers/My MacBook Pro/latam-challenge/src/q1_memory.py

Line #    Mem usage    Increment  Occurrences   Line Contents
     8    288.6 MiB    288.6 MiB           1   @memory_profiler.profile
     9                                         def q1_memory(client: bigquery.Client) -> List[Tuple[datetime.date, str]]:
    10    288.6 MiB      0.0 MiB           1       query = """
    11                                               with
    12                                               top_dates as (
    13                                                 select CAST(date AS DATE) AS tweets_date,
    14                                                 count(*) AS conteo_tweets
    15                                                 from tweets_dataset.tweets
    16                                                 group by tweets_date
    17                                                 order by conteo_tweets desc
    18                                         

[(datetime.date(2021, 2, 12), 'RanbirS00614606'),
 (datetime.date(2021, 2, 13), 'MaanDee08215437'),
 (datetime.date(2021, 2, 17), 'RaaJVinderkaur'),
 (datetime.date(2021, 2, 16), 'jot__b'),
 (datetime.date(2021, 2, 14), 'rebelpacifist'),
 (datetime.date(2021, 2, 18), 'neetuanjle_nitu'),
 (datetime.date(2021, 2, 15), 'jot__b'),
 (datetime.date(2021, 2, 20), 'MangalJ23056160'),
 (datetime.date(2021, 2, 23), 'Surrypuria'),
 (datetime.date(2021, 2, 19), 'Preetm91')]

In [ ]:
import cProfile
from q1_time import q1_time
cProfile.run('q1_time(bigquery_client)')

         15198 function calls (15126 primitive calls) in 0.996 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        3    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:404(parent)
       30    0.000    0.000    0.000    0.000 <string>:1(<lambda>)
        1    0.000    0.000    0.996    0.996 <string>:1(<module>)
        3    0.000    0.000    0.000    0.000 __init__.py:121(extra_headers)
        6    0.000    0.000    0.000    0.000 __init__.py:144(http)
        3    0.000    0.000    0.000    0.000 __init__.py:1455(debug)
        3    0.000    0.000    0.000    0.000 __init__.py:1724(isEnabledFor)
        1    0.000    0.000    0.000    0.000 __init__.py:183(dumps)
        3    0.000    0.000    0.000    0.000 __init__.py:187(get_api_base_url_for_mtls)
        6    0.000    0.000    0.000    0.000 __init__.py:203(_http)
       10    0.000    0.000    0.000    0.000 __init__.py:211(_date_from_iso8601_date

## Question 02

In [ ]:
from q2_memory import q2_memory
q2_memory(bigquery_client)

Filename: /content/drive/Othercomputers/My MacBook Pro/latam-challenge/src/q2_memory.py

Line #    Mem usage    Increment  Occurrences   Line Contents
     8    289.3 MiB    289.3 MiB           1   @memory_profiler.profile
     9                                         def q2_memory(client: bigquery.Client) -> List[Tuple[datetime.date, str]]:
    10    289.3 MiB      0.0 MiB           1       query = r"""
    11                                               SELECT
    12                                                   emoji AS emoji,
    13                                                   CAST(COUNT(*) AS INT) AS count
    14                                               FROM (
    15                                                   SELECT
    16                                                       REGEXP_EXTRACT_ALL(content, r"(?:[\x{1F300}-\x{1F5FF}]|[\x{1F900}-\x{1F9FF}]|[\x{1F600}-\x{1F64F}]|[\x{1F680}-\x{1F6FF}]|[\x{2600}-\x{26FF}]\x{FE0F}?|[\x{2700}-\x{27BF}]\x{FE0F}?|\x{24C

[('✊', 2402),
 ('❤️', 1382),
 ('❤', 397),
 ('☮️', 316),
 ('♂️', 179),
 ('✌️', 168),
 ('♀️', 148),
 ('✌', 106),
 ('‼️', 74),
 ('♥️', 73)]

In [ ]:
import cProfile
from q2_time import q2_time
cProfile.run('q2_time(bigquery_client)')

         15015 function calls (14943 primitive calls) in 0.778 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        3    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:404(parent)
       30    0.000    0.000    0.000    0.000 <string>:1(<lambda>)
        1    0.000    0.000    0.778    0.778 <string>:1(<module>)
        3    0.000    0.000    0.000    0.000 __init__.py:121(extra_headers)
        6    0.000    0.000    0.000    0.000 __init__.py:144(http)
        3    0.000    0.000    0.000    0.000 __init__.py:1455(debug)
        3    0.000    0.000    0.000    0.000 __init__.py:1724(isEnabledFor)
        1    0.000    0.000    0.000    0.000 __init__.py:183(dumps)
        3    0.000    0.000    0.000    0.000 __init__.py:187(get_api_base_url_for_mtls)
        6    0.000    0.000    0.000    0.000 __init__.py:203(_http)
        3    0.000    0.000    0.000    0.000 __init__.py:227(build_api_url)
        

# Question 03:

In [ ]:
from q3_memory import q3_memory
q3_memory(bigquery_client)

Filename: /content/drive/Othercomputers/My MacBook Pro/latam-challenge/src/q3_memory.py

Line #    Mem usage    Increment  Occurrences   Line Contents
     8    231.8 MiB    231.8 MiB           1   @memory_profiler.profile
     9                                         def q3_memory(client: bigquery.Client) -> List[Tuple[datetime.date, str]]:
    10    231.8 MiB      0.0 MiB           1       query = """
    11                                               SELECT user.username, count(*) AS conteo_menciones
    12                                               FROM tweets_dataset.tweets,
    13                                               UNNEST(mentionedUsers) AS user
    14                                               group by user.username
    15                                               order by conteo_menciones desc
    16                                               LIMIT 10
    17                                               """
    18    231.8 MiB      0.0 MiB           1

[('narendramodi', 2265),
 ('Kisanektamorcha', 1840),
 ('RakeshTikaitBKU', 1644),
 ('PMOIndia', 1427),
 ('RahulGandhi', 1146),
 ('GretaThunberg', 1048),
 ('RaviSinghKA', 1019),
 ('rihanna', 986),
 ('UNHumanRights', 962),
 ('meenaharris', 926)]

In [ ]:
import cProfile
from q3_time import q3_time
cProfile.run('q3_time(bigquery_client)')

         14899 function calls (14827 primitive calls) in 0.831 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        3    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:404(parent)
       30    0.000    0.000    0.000    0.000 <string>:1(<lambda>)
        1    0.000    0.000    0.840    0.840 <string>:1(<module>)
        3    0.000    0.000    0.000    0.000 __init__.py:121(extra_headers)
        6    0.000    0.000    0.000    0.000 __init__.py:144(http)
        3    0.000    0.000    0.000    0.000 __init__.py:1455(debug)
        3    0.000    0.000    0.000    0.000 __init__.py:1724(isEnabledFor)
        1    0.000    0.000    0.000    0.000 __init__.py:183(dumps)
        3    0.000    0.000    0.000    0.000 __init__.py:187(get_api_base_url_for_mtls)
        6    0.000    0.000    0.000    0.000 __init__.py:203(_http)
        3    0.000    0.000    0.001    0.000 __init__.py:227(build_api_url)
        